# Econ 8210 Quant Macro, Homework 1
## Part 2 - Solution Methods
Haosi Shen, Fall 2024

In [1]:
# Housekeeping
import numpy as np
import pandas as pd
import time
import torch

np.random.seed(42) 

## Computing Pareto Efficient Allocations

Consider an endowment economy with $m$ different goods and $n$ agents. Each agent $i = 1, ..., n$ has an endowment $e_j^i >0$ for every $j = 1, ..., m$ and a utility function of the form
$$ u^i (x) = \sum_{j=1}^{m} \alpha_j \frac{x_{j}^{1+\omega_j^i}}{1+\omega_j^i} $$
where $\alpha_j > 0 > \omega_j^i$ are agent-specific parameters.

Given some social weights $\lambda_i > 0$, solve for the social planner’s problem for $m = n = 3$ using the **Adam (Adaptive Moment Estimation)** method. Try different values of $\alpha_j,\;\omega_j^i,\;\lambda_i$. 

$$ \max_{\{x^i\}} \; \sum_{i=1}^{n} \lambda_i u^{i}(x)$$

Compute first the case where all the agents have the same parameters and
social weights and later a case where there is a fair degree of heterogeneity.

How does the method perform? How does heterogeneity in the agent-specific parameters
affect the results?

Can you handle the case where $m = n = 10$?

> I choose to use **Adam** since it is more efficient for high-dimensional optimization problems and offers more stability and robustness. However, if we are only solving for the case of $m=n=3$, then the Newton-Raphson method might be more ideal since this problem is relatively smooth. Adam only requires gradient information and does not involve inverting the Hessian.

In [2]:
# utility function for agent i
def utility(x, alpha, omega):
    return torch.sum(alpha * (x ** (1 + omega)) / (1 + omega))


# Incorporate resource constraints for each good j
# Define total endowments
endowments = torch.tensor([30.0, 30.0, 30.0], dtype = torch.float32)
# endowments = torch.tensor([10.0, 20.0, 40.0], dtype = torch.float32)

### Case I: Homogeneous Agents

> $m = n = 3$

All agents $j$ have the same parameters $\alpha_j, \omega_j^i$ and social weights $\lambda_j$. 

In [3]:
# Social planner's objective, with Homogeneous Agents
def social_planner_objective_homog(x, alpha_j, omega_j, lambda_i, endowments, penalty_weight = 1000):
    total_utility = 0
    total_allocations = torch.zeros(m)

    for i in range(n):
        x_i_softplus = torch.nn.functional.softplus(x[i]) # ensure nonneg allocations
        total_utility += lambda_i[i] * utility(x_i_softplus, alpha_j, omega_j)
        total_allocations += x_i_softplus

    # penalize if RC is violated, i.e. total allocations > endowments
    penalty = torch.sum(torch.clamp(total_allocations - endowments, min = 0) ** 2)

    return -total_utility + penalty_weight * penalty   # maximization, so take negative

In [4]:
import torch.optim as optim

# =============== DEFINE PARAMETERS ===============
m, n = 3, 3  # 3 goods, 3 agents
alpha_j = torch.tensor([1.0, 1.0, 1.0], dtype = torch.float32) 
omega_j = torch.tensor([[0.5, 0.5, 0.5]] * n, dtype = torch.float32)
lambda_i = torch.tensor([1.0, 1.0, 1.0], dtype = torch.float32)  # Pareto weights

# initial allocations (using small positive values)
x_i = torch.rand((n, m), requires_grad = True)

# Set up optimizer
optimizer = optim.Adam([x_i], lr = 0.01)

# Optimization
num_iterations = 1000
for iteration in range(num_iterations):
    optimizer.zero_grad()
    objective = social_planner_objective_homog(x_i, alpha_j, omega_j, lambda_i, endowments)
    objective.backward()
    optimizer.step()

    if (iteration+1) % 100 == 0:
        print(f"Iteration {iteration}: Objective = {-objective.item()}")


# Final optimal allocations
final_allocations = torch.nn.functional.softplus(x_i).detach().numpy()

df_final_homog = pd.DataFrame(
    final_allocations,
    index = [f'Agent {i+1}' for i in range(final_allocations.shape[0])],
    columns = [f'Good {j+1}' for j in range(final_allocations.shape[1])]
)

print("\n Optimal allocations for Homogeneous Agents:")
print(df_final_homog)


Iteration 99: Objective = 40.35831069946289
Iteration 199: Objective = 88.05348205566406
Iteration 299: Objective = 156.2135772705078
Iteration 399: Objective = 238.33685302734375
Iteration 499: Objective = 330.75048828125
Iteration 599: Objective = 431.65130615234375
Iteration 699: Objective = 539.98388671875
Iteration 799: Objective = 569.2763061523438
Iteration 899: Objective = 569.2817993164062
Iteration 999: Objective = 569.2822265625

 Optimal allocations for Homogeneous Agents:
            Good 1     Good 2     Good 3
Agent 1  10.044940   9.975189  10.050016
Agent 2   9.990323   9.937491   9.998878
Agent 3   9.969482  10.092060   9.955850


### Case II: Heterogeneous Agents 
> $m = n = 3$

* Each agent $i$ has their own set of $\alpha_j, \omega_j^i$ parameters.
* Pareto weights $\lambda_i$ differ among agents.
* Resource constraint remains the same. Total endowment for each good is 30.

In [5]:
# Social planner's objective, with Heterogeneous Agents

def social_planner_objective_heterog(x, alpha_j, omega_j, lambda_i, endowments, penalty_weight = 1000):
    total_utility = 0
    total_allocations = torch.zeros(m)

    for i in range(n):
        x_i_softplus = torch.nn.functional.softplus(x[i]) # ensure nonneg allocations
        total_utility += lambda_i[i] * utility(x_i_softplus, alpha_j[i], omega_j[i])
        total_allocations += x_i_softplus

    # penalize if RC is violated, i.e. total allocations > endowments
    penalty = torch.sum(torch.clamp(total_allocations - endowments, min = 0) ** 2)

    return -total_utility + penalty_weight * penalty   # maximization, so take negative


In [6]:
# =============== DEFINE PARAMETERS ===============
alpha_j = torch.tensor([[1.0, 0.8, 1.2],  # agent 1
                        [1.1, 0.9, 1.3],  # agent 2
                        [0.9, 0.7, 1.1]])  # agent 3

omega_j = torch.tensor([[0.3, 0.5, 0.7], 
                        [0.4, 0.6, 0.8],  
                        [0.5, 0.4, 0.6]]) 

lambda_i = torch.tensor([0.9, 1.1, 1.0])  # Social weights


# initial allocations (using small positive values)
x_i = torch.rand((n, m), requires_grad = True)
#x_i = (torch.rand((n, m), requires_grad=True) * endowments / n).clone().detach().requires_grad_(True)

# Set up optimizer
optimizer = optim.Adam([x_i], lr = 0.001)

# Optimization
num_iterations = 10000
for iteration in range(num_iterations):
    optimizer.zero_grad()
    objective = social_planner_objective_heterog(x_i, alpha_j, omega_j, lambda_i, endowments)
    objective.backward()
    optimizer.step()

    if (iteration+1) % 1000 == 0:
        print(f"Iteration {iteration}: Objective = {-objective.item()}")


# Final optimal allocations
final_allocations = torch.nn.functional.softplus(x_i).detach().numpy()

df_final_heterog = pd.DataFrame(
    final_allocations,
    index = [f'Agent {i+1}' for i in range(final_allocations.shape[0])],
    columns = [f'Good {j+1}' for j in range(final_allocations.shape[1])]
)

print("\n Optimal allocations for Heterogeneous Agents:")
print(df_final_heterog)

Iteration 999: Objective = 16.520526885986328
Iteration 1999: Objective = 33.85341262817383
Iteration 2999: Objective = 56.39704513549805
Iteration 3999: Objective = 82.38998413085938
Iteration 4999: Objective = 111.12158966064453
Iteration 5999: Objective = 142.30734252929688
Iteration 6999: Objective = 175.8013916015625
Iteration 7999: Objective = 210.39227294921875
Iteration 8999: Objective = 220.77528381347656
Iteration 9999: Objective = 227.41481018066406

 Optimal allocations for Heterogeneous Agents:
            Good 1     Good 2     Good 3
Agent 1   9.294359  10.029044   9.890506
Agent 2  10.500991  11.101754  11.470001
Agent 3  10.205898   8.870392   8.642317


> The `Adam` optimizer performs fairly well for both the homogeneous and heterogeneous agents cases. The computations are fast and produce sufficiently accurate results. 
> 
> Introducing heterogeneity in the agent-specific parameters does affect the socially optimal allocations, and significantly slow down convergence of optimization.

### Case III: 10 Agents, 10 Goods

> $m=n=10$

Since the `Adam` optimizer generally works well for higher-dimension problems, we now try computing the Pareto efficient allocations of a 10-agent 10-good economy. 

**Consider homogeneity across agents.**

In [7]:
m, n = 10, 10  # 10 goods, 10 agents
endowments = torch.tensor([30.0] * m)  # total endowments for 10 goods

alpha_j = torch.tensor([1.0] * m)  
omega_j = torch.tensor([0.5] * m)  
lambda_i = torch.tensor([1.0] * n)

x_i = torch.rand((n, m), requires_grad = True)  # initial allocations

optimizer = optim.Adam([x_i], lr = 0.01)

# Optimization
num_iterations = 1000
for iteration in range(num_iterations):
    optimizer.zero_grad()
    objective = social_planner_objective_homog(x_i, alpha_j, omega_j, lambda_i, endowments)
    objective.backward()
    optimizer.step()

    if (iteration+1) % 100 == 0:
        print(f"Iteration {iteration}: Objective = {-objective.item()}")


# Final optimal allocations
final_allocations = torch.nn.functional.softplus(x_i).detach().numpy()

df_final_homog = pd.DataFrame(
    final_allocations,
    index = [f'Agent {i+1}' for i in range(final_allocations.shape[0])],
    columns = [f'Good {j+1}' for j in range(final_allocations.shape[1])]
)

print("\n Optimal allocations for Homogeneous Agents:")
print(df_final_homog)

Iteration 99: Objective = 154.7710723876953
Iteration 199: Objective = 332.29345703125
Iteration 299: Objective = 345.7464599609375
Iteration 399: Objective = 347.0973205566406
Iteration 499: Objective = 347.120361328125
Iteration 599: Objective = 347.1561584472656
Iteration 699: Objective = 346.99609375
Iteration 799: Objective = 346.80059814453125
Iteration 899: Objective = 347.25238037109375
Iteration 999: Objective = 346.9298095703125

 Optimal allocations for Homogeneous Agents:
            Good 1    Good 2    Good 3    Good 4    Good 5    Good 6  \
Agent 1   2.914130  3.263182  2.595877  2.866040  2.661801  3.439897   
Agent 2   3.327475  3.326041  2.884481  3.391286  3.429351  2.896719   
Agent 3   2.730921  3.062123  3.117459  2.966691  3.372595  2.932783   
Agent 4   2.764751  3.115304  3.232926  3.463181  2.797698  3.470091   
Agent 5   2.794737  2.827952  3.373684  2.926597  2.746746  2.615396   
Agent 6   3.634638  3.031342  3.408268  2.973178  2.690806  2.633177   
Agent 7

**Now, try the heterogeneous agents case.**

In [8]:
# Heterogeneous agent-specific parameters
alpha_j = torch.tensor([
    [1.0, 0.8, 1.2, 0.9, 1.1, 0.7, 1.3, 0.95, 1.05, 1.15] for _ in range(n)]) 

omega_j = torch.tensor([
    [0.3, 0.4, 0.5, 0.35, 0.45, 0.55, 0.25, 0.6, 0.65, 0.5] for _ in range(n)])

lambda_i = torch.tensor([0.9, 1.1, 0.8, 1.2, 1.0, 0.85, 1.15, 0.95, 1.05, 1.0])


In [9]:
x_i = torch.rand((n, m), requires_grad=True)
optimizer = optim.Adam([x_i], lr = 0.001)

# Optimization
num_iterations = 10000
for iteration in range(num_iterations):
    optimizer.zero_grad()
    objective = social_planner_objective_heterog(x_i, alpha_j, omega_j, lambda_i, endowments)
    objective.backward()
    optimizer.step()

    if (iteration+1) % 1000 == 0:
        print(f"Iteration {iteration}: Objective = {-objective.item()}")


# Final optimal allocations
final_allocations = torch.nn.functional.softplus(x_i).detach().numpy()

df_final_heterog = pd.DataFrame(
    final_allocations,
    index = [f'Agent {i+1}' for i in range(final_allocations.shape[0])],
    columns = [f'Good {j+1}' for j in range(final_allocations.shape[1])]
)

print("\n Optimal allocations for Heterogeneous Agents:")
print(df_final_heterog)

Iteration 999: Objective = 169.78448486328125
Iteration 1999: Objective = 333.05609130859375
Iteration 2999: Objective = 349.08544921875
Iteration 3999: Objective = 357.8846740722656
Iteration 4999: Objective = 380.14593505859375
Iteration 5999: Objective = 423.39337158203125
Iteration 6999: Objective = 482.5692138671875
Iteration 7999: Objective = 535.2533569335938
Iteration 8999: Objective = 575.578857421875
Iteration 9999: Objective = 611.5863647460938

 Optimal allocations for Heterogeneous Agents:
            Good 1    Good 2    Good 3    Good 4    Good 5    Good 6  \
Agent 1   0.117886  0.152617  0.065931  0.127449  0.081844  0.062376   
Agent 2   7.421418  7.486674  7.738941  8.932431  8.769207  8.761271   
Agent 3   0.107674  0.107837  0.043850  0.093451  0.042553  0.059907   
Agent 4   8.832698  9.452785  9.899761  8.762597  9.390129  9.841595   
Agent 5   0.428526  0.206919  0.275376  0.326674  0.179943  0.065967   
Agent 6   0.168618  0.161534  0.052743  0.141985  0.047812  

> The method works well for $m=n=10$, even after we introduce household heterogeneity. 

## Computing Equilibrium Allocations

Using the same model as in the previous exercise. Find the equilibrium prices for each good $p^j$.

1. Solve for the first-order conditions of each agent.
1. Aggregate the excess demands. 
1. Solve the resulting system of nonlinear equations, when all markets clear.

### Homogeneous Agents, $m=n=3$

**FOCs**
> To find the demand for each good, we solve the Lagrangean:
> $$\mathcal{L}=\sum_{j=1}^{m}\alpha_j \cdot \frac{x_{ij}^{1+\omega}}{1+\omega}-\lambda_i \Big(\sum_{j=1}^{m}p_j\cdot x_{ij} - Y_i\Big)$$
> where $Y_i$ is the income of agent $i$. 
> The FOCs for this problem yield
> $$ x_{ij} = \frac{\alpha_j \cdot p_{j}^{-\omega}}{\sum_{k=1}^{m} \alpha_k \cdot p_{k}^{-\omega} }\cdot Y_i $$

**Excess Demand**
> Aggregate the demand across agents and compare it to the total endowment to find the excess demand for each good
> $$z_j(p) =  \sum_{i=1}^{n} x_{ij}(p) - e_j$$
> Markets clear when $z_j(p)=0$ for all goods $j$.

**Equilibrium Prices**
> The goal is to find prices $p$ such that the excess demand is zero for all goods. This involves numerically solving a system of nonlinear equations.

In [10]:
from scipy.optimize import fsolve

# Step 1: Solve for individual agent's demand from FOC

def agent_demand(p, alpha, omega, income):
    """
    - p: prices for each good (vector of size m)
    - alpha: preference weights (vector of size m)
    - omega: curvature parameter (scalar)
    - income: agent's income (scalar)

    Returns: demand for each good (vector of size m)
    """
    return (alpha * (p ** -omega)) / (np.sum(alpha * (p ** -omega))) * income



# Step 2: Solve for excess demand for each good

def excess_demand(p, endowments, alpha, omega, incomes):
    """
    - endowments: total endowment for each good (vector of size m)
    - incomes: total income for each agent (vector of size n)

    Returns: excess demand for each good (vector of size m)
    """
    n, m = incomes.shape[0], endowments.shape[0]
    total_demand = np.zeros(m)
    
    for i in range(n):
        demand_i = agent_demand(p, alpha, omega, incomes[i])
        total_demand += demand_i
    
    return total_demand - endowments 


In [11]:
# Step 3: Solve for equilibrium prices where excess demand = 0
def find_eq_prices(endowments, alpha, omega, incomes):
    """
    Solve for eq prices using `fsolve` to find root of excess demand function.
    """
    p0 = np.ones(endowments.shape[0])   # Initial guess
    
    eq_prices = fsolve(lambda p: excess_demand(p, endowments, alpha, omega, incomes), p0)
    
    return eq_prices


In [12]:
# =============== DEFINE PARAMETERS (homogeneity) ===============
m, n = 3, 3  # 3 goods, 3 agents
alpha_j = np.ones(m)  # preference weights
omega_j = 0.5  # curvature param
endowments = np.array([30.0, 30.0, 30.0])  # total endowments
incomes = np.ones(n) * np.sum(endowments) / n  # equally distributed income

eq_prices = find_eq_prices(endowments, alpha_j, omega_j, incomes)
print(eq_prices)

[1. 1. 1.]


By the FWT, every competitive equilibrium is Pareto efficient. Equilibrium prices align with marginal rates of substitution, and budget constraints enforce resource constraints. Therefore, the Arrow-Debreu equilibrium allocations coincide with the Pareto-efficient allocations.

### Heterogeneous Agents, $m=n=3$


**FOCs**
> To find the demand for each good, we solve the Lagrangean:
> $$\mathcal{L}=\sum_{j=1}^{m}\alpha_j^i \cdot \frac{x_{ij}^{1+\omega_j^i}}{1+\omega_j^i}-\lambda_i \Big(\sum_{j=1}^{m}p_j\cdot x_{ij} - Y_i\Big)$$
> where $Y_i$ is the income of agent $i$. 
> The FOCs for this problem yield
> $$ x_{ij} = \frac{\alpha_j^i \cdot p_{j}^{-\omega_j^i}}{\sum_{k=1}^{m} \alpha_k^i \cdot p_{k}^{-\omega_j^i} }\cdot Y_i $$

**Excess Demand**
> Aggregate the demand across agents and compare it to the total endowment to find the excess demand for each good
> $$z_j(p) =  \sum_{i=1}^{n} x_{ij}(p) - e_j$$
> Markets clear when $z_j(p)=0$ for all goods $j$.

**Equilibrium Prices**
> The goal is to find prices $p$ such that the excess demand is zero for all goods. This involves numerically solving a system of nonlinear equations.